# Nonlinear problems
We want to solve a nonlinear PDE. 

## A simple scalar PDE
We consider the simple PDE 

$$
- \Delta u + 3 u^3 = 1 \text{ in } \Omega
$$

on the unit square $\Omega = (0,1)^2$. 

We note that this PDE can also be formulated as a nonlinear minimization problem 
(cf. [unit 3.6](unit-3.6-nonlmin/nonlmin.ipynb) ).

In [ ]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *

mesh = Mesh (unit_square.GenerateMesh(maxh=0.3))

In NGSolve we can solve the PDE conveniently using the *linearization* feature of `SymbolicBFI`.

The `BilinearForm` (which is not bilinear!) needed in the weak formulation is
$$
  A(u,v) = \int_{\Omega} \nabla u \nabla v + 3 u^3 v - 1 v ~ dx \quad ( = 0 ~ \forall~v \in H^1_0)
$$

In [ ]:
V = H1(mesh, order=3, dirichlet=[1,2,3,4])
u,v = V.TnT()
a = BilinearForm(V)
a += SymbolicBFI( grad(u) * grad(v) + 5*u*u*v- 1 * v)

### Newton's method

We use Newton's method and make the loop:
* Given an initial guess $u^0$
* loop over $i=0,..$ until convergence:
  * Compute linearization: $A u^i + \delta A(u^i) \Delta u^{i} = 0 $:
    * $f^i = A u^i$ 
    * $B^i = \delta A(u^i)$ 
    * Solve $B^i \Delta u^i = -f^i$
  * Update $u^{i+1} = u^i + \Delta u^{i}$
  * Evaluate stopping criteria

As a stopping criteria we take $ \langle A u^i,\Delta u^i \rangle = \langle A u^i, A u^i \rangle_{(B^i)^{-1}}< \varepsilon$.  

In [ ]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break

In [ ]:
gfu = GridFunction(V)
Draw(gfu,mesh,"u")
SimpleNewtonSolve(gfu,a)

## A trivial problem:

$$
  5 u^2 = 1, \qquad u \in \mathbb{R}.
$$

In [ ]:
V = NumberSpace(mesh)
u,v = V.TnT()
a = BilinearForm(V)
a += SymbolicBFI( 5*u*u*v - 1 * v)
gfu = GridFunction(V)
gfu.vec[:] = 1
SimpleNewtonSolve(gfu,a)

print("\nscalar solution", gfu.vec[0], "(exact: ", sqrt(0.2), ")")

## Another example: Stationary Navier-Stokes:
Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$, $\lambda \in \mathbb{R}$ so that
\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} + (\mathbf{u} \cdot \nabla) \mathbf{u} \cdot \mathbf{v}& - \int_{\Omega} \operatorname{div}(\mathbf{v}) p & &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q & & 
+ \int_{\Omega} \lambda q
&= 0 && \forall q \in Q, \\
& \int_{\Omega} \mu p & &= 0 && \forall \mu \in \mathbb{R}.
\end{align}



In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.05))
V = VectorH1(mesh,order=3, dirichlet="bottom|right|top|left")
Q = H1(mesh,order=2)
N = NumberSpace(mesh)
X = FESpace([V,Q,N])
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
nu = Parameter(1)
# Note: grad(u) is a matrix where each row is the gradient of one of the components
a += SymbolicBFI(nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)-div(u)*q-div(v)*p-lam*q-mu*p)
gfu = GridFunction(X)
gfu.components[0].Set(CoefficientFunction((4*x*(1-x),0)),definedon=mesh.Boundaries("top"))

In [ ]:
SimpleNewtonSolve(gfu,a)
Draw(gfu.components[1],mesh,"p")
Draw(gfu.components[0],mesh,"u")

In [ ]:
nu.Set(0.01)
SimpleNewtonSolve(gfu,a)
Redraw()